In [4690]:
import pandas as pd
import re
pd.set_option('display.max_rows' , 200)

In [4691]:
company_name = 'tcs'.upper()

In [4692]:
particulars = 'particulars'

In [4693]:
share_price_in = 1000000

In [4694]:
def calculate_growth(current , previous) :
    if current == 0 and previous == 0 or current == 0 and previous != 0:
        return 0
    if current != 0 and previous == 0 :
        return current / 100
    return ((current/previous) - 1)
       

In [4695]:
def calculate_growth_series(curr_df,prev_df):
    temp_df = curr_df.copy()
    for index in curr_df.index :
        temp_df.loc[index] = calculate_growth(curr_df.loc[index],prev_df[index])
    return temp_df
    

In [4696]:
def get_share_price_in(price) :
    return round((price / share_price_in),2)

    

In [4697]:
def vertical_analysis(item , divider) :
    return item/divider
    

In [4698]:
def round_off_to(num , limit) :
    return round(num,limit)

In [4699]:
def count_leading_space(s): 
    match = re.search(r"^\s*", s) 
    return 0 if not match else match.end()


In [4700]:
def getColumnFormatterbyPercentage(type) :
    columns_perc_formats = {}
    if type == 'income_statement' :
        for column in income_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'balance_sheet' :
        for column in balance_sheet.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'cashflow_statement' :
        for column in cashflow_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    return columns_perc_formats

In [4701]:
def calculate_ratio(dividend , divisor) : 
    return dividend.div(divisor.values).apply(lambda x: round(x,2))

In [4702]:
def styles(color = 'white',backcolor = '#308D46'):
    return [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%"),
                              ("color", color),
                              ("background", backcolor),
                              ("font-weight", '600'),
                              ("text-transform", 'uppercase')
                             ])]

In [4703]:
def days_turnover(ratio) :
    if ratio == '-' :
        return '-'
    return round((365 / ratio),2)

In [4704]:
def get_df_items(df , key_name , start_col) :
     filter = df[particulars].apply(lambda x: x.strip().lower()).eq(key_name.lower())
     temp_df = pd.DataFrame(df.loc[filter,start_col:])
     temp_df.reset_index(drop=True,inplace=True) 
     return temp_df

In [4705]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [4706]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [4707]:
income_statement_temp_columns = income_statement.columns.values
income_statement_temp_columns[0] = particulars
income_statement.columns = income_statement_temp_columns

balance_sheet_temp_columns = balance_sheet.columns.values
balance_sheet_temp_columns[0] = particulars
balance_sheet.columns = balance_sheet_temp_columns

cashflow_statement_temp_columns = cashflow_statement.columns.values
cashflow_statement_temp_columns[0] = particulars
cashflow_statement.columns = cashflow_statement_temp_columns

In [4708]:
basic_eps_filter = income_statement[particulars].eq('Basic EPS')
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [4709]:
WASO_filter = income_statement[particulars].isin(['Basic Weighted Average Shares Outstanding','Diluted Weighted Average Shares Outstanding','Basic WASO','Diluted WASO'])
waso_pos = income_statement[WASO_filter].index

In [4710]:
income_statement.dropna(how='all',subset=income_statement.columns[1:], inplace=True)
balance_sheet.dropna(how='all',subset=balance_sheet.columns[1:], inplace=True)
cashflow_statement.dropna(how='all',subset=cashflow_statement.columns[1:], inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [4711]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [4712]:
temp_df = temp_df.apply(get_share_price_in)

In [4713]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [4714]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [4715]:
temp_revenue = income_statement.loc[1]
income_statement.loc[1] = income_statement.loc[0]
income_statement.loc[0] = temp_revenue

In [4716]:
total_rev = income_statement[particulars].index == 0
income_statement.loc[total_rev,particulars] = income_statement.loc[total_rev,particulars].values[0].strip() 

In [4717]:
is_highlighted_indexes = pd.DataFrame(income_statement.loc[0:basic_eps_pos - 1,particulars])
is_highlighted_indexes = is_highlighted_indexes[particulars].apply(count_leading_space).isin([0,4])
is_highlighted_indexes = is_highlighted_indexes.loc[is_highlighted_indexes].index

In [4718]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [4719]:
total_equity_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Equity')
total_equity_row = balance_sheet.loc[total_equity_filter,start_col_balance_sheet:].index.values[0]
total_assets_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Assets')
total_assets_row = balance_sheet.loc[total_assets_filter,start_col_balance_sheet:]
total_liabilities_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Liabilities')
total_liabilities_row = balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:].index.values[0]
balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:] = balance_sheet.loc[[total_equity_row,total_liabilities_row],start_col_balance_sheet:].sum().values

In [4720]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [4721]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns) 
vertical_analysis_is[particulars] = income_statement.loc[0:basic_eps_pos - 1,particulars].copy()

In [4722]:
total_revenue_filter = income_statement[particulars].apply(lambda x: x.strip()).eq('Total Revenue')
total_revenue = income_statement.loc[total_revenue_filter,start_col:]

In [4723]:
for column in vertical_analysis_is.columns[1:] : 
    vertical_analysis_is[column] = income_statement.loc[0:basic_eps_pos - 1,column].apply(vertical_analysis,divider=total_revenue[column])
vertical_analysis_is.loc[0,particulars] = vertical_analysis_is.loc[0,particulars].strip() 

In [4724]:
def row_style(row , list , key , color,bgcolor):
   if row[key] in list :
       return pd.Series('color:'+color+';background-color:'+bgcolor, row.index)
   else :
        return pd.Series('', row.index)

In [4725]:
fil = vertical_analysis_is.loc[0:basic_eps_pos - 1,particulars].apply(count_leading_space).eq(0)
templist = vertical_analysis_is.loc[fil,particulars].values
vertical_analysis_is_style = vertical_analysis_is
vertical_analysis_is_style = vertical_analysis_is_style.loc[is_highlighted_indexes].style.apply(row_style,list=templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " income statement vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 
fil = None
templist = None

In [4726]:
balance_sheet_va_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_va_results_till = balance_sheet[balance_sheet_va_results_till].index.values[0]
bs_va_temp = balance_sheet.loc[:balance_sheet_va_results_till]
bs_va_temp = pd.DataFrame(bs_va_temp)
filt = bs_va_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_va = pd.DataFrame(bs_va_temp[filt])
bs_va_temp = None
balance_sheet_va_results_till = None
filt = None

In [4727]:
for column in balance_sheet_va.columns[1:] :
    balance_sheet_va[column] = balance_sheet_va.loc[:,column].apply(vertical_analysis,divider=total_assets_row[column])

In [4728]:
bs_fil = balance_sheet_va[particulars].apply(count_leading_space).isin([0,4])
bs_templist = balance_sheet_va.loc[bs_fil,particulars].values
balance_sheet_va_style = balance_sheet_va
balance_sheet_va_style = balance_sheet_va_style.style.apply(row_style,list=bs_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil = None
bs_templist = None

In [4729]:
#vertical_analysis_is_style

In [4730]:
#balance_sheet_va_style

In [4731]:
income_statement_ha = pd.DataFrame(columns= income_statement.columns)

In [4732]:
income_statement_ha[particulars] = income_statement[particulars]

In [4733]:
income_statement_ha.fillna(0 , inplace=True)

In [4734]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0,4])
for index , column in enumerate(income_statement_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = income_statement.columns[index-1]
        income_statement_ha.loc[temp_ha_filt,column] = calculate_growth_series(income_statement.loc[temp_ha_filt,column],income_statement.loc[temp_ha_filt,prevColumn])
        

In [4735]:
income_statement_ha = income_statement_ha[temp_ha_filt]

In [4736]:
#income_statement_ha.replace([float('inf'), float('-inf')], None, inplace=True)
#income_statement_ha.fillna(0 , inplace=True)

In [4737]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0])
iha_templist = income_statement_ha.loc[temp_ha_filt,particulars].values
income_statement_ha_style = income_statement_ha
income_statement_ha_style = income_statement_ha_style.style.apply(row_style,list=iha_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " Income statement horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

In [4738]:
balance_sheet_ha_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_ha_results_till = balance_sheet[balance_sheet_ha_results_till].index.values[0]
bs_ha_temp = balance_sheet.loc[:balance_sheet_ha_results_till]
bs_ha_temp = pd.DataFrame(bs_ha_temp)
filt = bs_ha_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_ha = pd.DataFrame(bs_ha_temp[filt])

In [4739]:
for index , column in enumerate(balance_sheet_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = balance_sheet_ha.columns[index-1]
        balance_sheet_ha.loc[:,column] = calculate_growth_series(balance_sheet.loc[:,column],balance_sheet.loc[:,prevColumn])
balance_sheet_ha[start_col_balance_sheet] = 0.0
prevColumn = None

In [4740]:
bs_fil_ha = balance_sheet_ha[particulars].apply(count_leading_space).isin([0,4])
bs_templist_ha = balance_sheet_ha.loc[bs_fil_ha,particulars].values
balance_sheet_ha_style = balance_sheet_ha
balance_sheet_ha_style = balance_sheet_ha_style.style.apply(row_style,list=bs_templist_ha,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil_ha = None
bs_templist_ha = None

In [4741]:
cashflow_statement_filt = cashflow_statement[particulars].apply(count_leading_space).isin([0,8,12])
cash_flow_stmt_ha = pd.DataFrame(cashflow_statement[cashflow_statement_filt])
cash_flow_stmt_ha.reset_index(inplace=True, drop=True)
cash_flow_stmt_ha_temp = cash_flow_stmt_ha.copy()

In [4742]:
for index , column in enumerate(cash_flow_stmt_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = cash_flow_stmt_ha.columns[index-1]
        cash_flow_stmt_ha.loc[:,column] = calculate_growth_series(cash_flow_stmt_ha_temp.loc[:,column],cash_flow_stmt_ha_temp.loc[:,prevColumn])
cash_flow_stmt_ha[start_col_cashflow] = 0.0
cash_flow_stmt_ha_temp = None
cashflow_statement_filt = None
prevColumn = None

In [4743]:
cf_fil_ha = cash_flow_stmt_ha[particulars].apply(count_leading_space).isin([0])
cf_templist_ha = cash_flow_stmt_ha.loc[cf_fil_ha,particulars].values
cash_flow_stmt_ha_style = cash_flow_stmt_ha
cash_flow_stmt_ha_style = cash_flow_stmt_ha_style.style.apply(row_style,list=cf_templist_ha,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " cash flow horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('cashflow_statement'),precision=2) 
cf_fil_ha = None
cf_templist_ha = None

In [4744]:
#income_statement_ha_style

In [4745]:
#balance_sheet_ha_style

In [4746]:
#cash_flow_stmt_ha_style

In [4747]:
is_trend_base = income_statement[start_col]

In [4748]:
income_statement_trend = pd.DataFrame(columns=income_statement.columns)
income_statement_trend[particulars] = income_statement[particulars]
income_statement_trend[start_col] = 0
income_statement_trend.fillna(0,inplace=True)

In [4749]:
for index , column in enumerate(income_statement_trend.columns) :
    if index < 2 :
        pass;
    else:
        income_statement_trend.loc[:,column] = calculate_growth_series(income_statement.loc[:,column],is_trend_base)
is_trend_base = None      

In [4750]:
temp_ta_filt = income_statement_trend[particulars].apply(count_leading_space).isin([0])
ita_templist = income_statement_trend.loc[temp_ta_filt,particulars].values
income_statement_trend_style = income_statement_trend
income_statement_trend_style = income_statement_trend_style.style.apply(row_style,list=ita_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " Income statement trend analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

In [4751]:
temp_balance = balance_sheet.loc[:balance_sheet_ha_results_till]

In [4752]:
bs_trend_base = temp_balance[start_col_balance_sheet]
balance_sheet_trend = pd.DataFrame(columns=temp_balance.columns)
balance_sheet_trend[particulars] = temp_balance[particulars]
balance_sheet_trend[start_col] = 0
balance_sheet_trend.fillna(0,inplace=True)

In [4753]:
for index , column in enumerate(balance_sheet_trend.columns) :
    if index < 2 :
        pass;
    else:
        balance_sheet_trend.loc[:,column] = calculate_growth_series(temp_balance.loc[:,column],bs_trend_base)
temp_balance = None
bs_trend_base = None

In [4754]:
bs_fil_ta = balance_sheet_trend[particulars].apply(count_leading_space).isin([0,4])
bs_templist_ta = balance_sheet_trend.loc[bs_fil_ta,particulars].values
balance_sheet_trend_style = balance_sheet_trend
balance_sheet_trend_style = balance_sheet_trend_style.style.apply(row_style,list=bs_templist_ta,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet trend analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil_ta = None
bs_templist_ta = None

In [4755]:
cashflow_statement_filt = cashflow_statement[particulars].apply(count_leading_space).isin([0,8,12])
cash_flow_stmt_trend = pd.DataFrame(columns=cashflow_statement.columns)
cash_flow_stmt_trend_temp = cashflow_statement[cashflow_statement_filt].copy()
cash_flow_stmt_trend_temp.reset_index(inplace=True,drop=True)
cash_flow_stmt_trend[particulars] = cash_flow_stmt_trend_temp[particulars]
cash_flow_stmt_trend[start_col_cashflow] = 0

In [4756]:
cf_trend_base = cash_flow_stmt_trend_temp[start_col_cashflow]

In [4757]:
for index , column in enumerate(cash_flow_stmt_trend.columns) :
    if index < 2 :
        pass;
    else:
        cash_flow_stmt_trend.loc[:,column] = calculate_growth_series(cash_flow_stmt_trend_temp.loc[:,column],cf_trend_base)
cash_flow_stmt_trend_temp = None
cf_trend_base = None

In [4758]:
cf_fil_ta = cash_flow_stmt_trend[particulars].apply(count_leading_space).isin([0])
cf_templist_ta = cash_flow_stmt_trend.loc[cf_fil_ta,particulars].values
cash_flow_stmt_trend_style = cash_flow_stmt_trend
cash_flow_stmt_trend_style = cash_flow_stmt_trend.style.apply(row_style,list=cf_templist_ta,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " cash flow trend analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('cashflow_statement'),precision=2) 
cf_fil_ta = None
cf_templist_ta = None

In [4759]:
#income_statement_trend_style

In [4760]:
#balance_sheet_trend_style

In [4761]:
#cash_flow_stmt_trend_style

In [4762]:
current_assets_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('total current assets')
current_liabilities_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('total current liabilities')

In [4763]:
current_assets = balance_sheet.loc[current_assets_filter,start_col_balance_sheet:].copy()
current_liabilities = balance_sheet.loc[current_liabilities_filter,start_col_balance_sheet:].copy()

In [4764]:
solvency_ratio = pd.DataFrame(columns=balance_sheet.columns)

In [4765]:
solvency_ratio[particulars] = ['current ratio' , 'quick ratio', 'cash ratio']

In [4766]:
solvency_ratio.loc[0:0,start_col_balance_sheet:] = calculate_ratio(current_assets,current_liabilities).values

In [4767]:
inventories_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('inventories')

In [4768]:
inventories = balance_sheet.loc[inventories_filter,start_col_balance_sheet:].copy()
if len(inventories.value_counts().values) == 0 :
    data = [[0] * len(inventories.columns)]
    inventories = pd.DataFrame(data=data , columns=inventories.columns).copy()
quick_assets = current_assets - inventories.values

In [4769]:
solvency_ratio.loc[1:1,start_col_balance_sheet:] =  calculate_ratio(quick_assets,current_liabilities).values

In [4770]:
cash_and_ceq_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('cash and cash equivalents')

In [4771]:
cash_and_ceq = balance_sheet.loc[cash_and_ceq_filter,start_col_balance_sheet:].copy()

In [4772]:
solvency_ratio.loc[2:2,start_col_balance_sheet:] = calculate_ratio(cash_and_ceq,current_liabilities).values

In [4773]:
solvency_ratio

,particulars,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,current ratio,2.67,2.74,2.4,2.87,5.55,4.56,4.17,3.33,2.91,2.56
1,quick ratio,2.66,2.74,2.4,2.87,5.55,4.55,4.17,3.33,2.91,2.56
2,cash ratio,0.57,0.92,0.9,0.29,0.25,0.27,0.33,0.32,0.2,0.29


In [4774]:
turnover_ratio = pd.DataFrame(columns=balance_sheet.columns)

In [4775]:
turnover_ratio[particulars] = ['Receivable turnover' , 'Inventory turnover', 'Payables turnover']

In [4776]:
#turnover_ratio.drop([start_col_balance_sheet] , axis=1 , inplace=True)

In [4777]:
account_receivables = get_df_items(balance_sheet,'Trade and Other Receivables, Current' , start_col_balance_sheet)

In [4778]:
invetory = get_df_items(balance_sheet,'Inventories' , start_col_balance_sheet)

In [4779]:
cogs = get_df_items(income_statement,'Cost of Revenue' , start_col).apply(abs)

In [4780]:
for i , column in enumerate(balance_sheet.columns):
    if i > 1 :
        prevColumn = balance_sheet.columns[i - 1]
        ar_mean = account_receivables.loc[:,prevColumn:column].mean(axis=1)
        turnover_ratio.loc[0:0,column] = calculate_ratio(total_revenue.loc[:,column],ar_mean).values


In [4781]:
for i , column in enumerate(balance_sheet.columns):
    if(len(invetory.value_counts()) == 0) :
        turnover_ratio.loc[1:1,start_col_balance_sheet:] = '-'
        break
    elif i > 1 :
        prevColumn = balance_sheet.columns[i - 1]
        inv_mean = invetory.loc[:,prevColumn:column].mean(axis=1)
        turnover_ratio.loc[1:1,column] = calculate_ratio(cogs.loc[:,column],inv_mean).values    

In [4782]:
bl_is_col_diff = len(balance_sheet.columns) - len(income_statement.columns)

In [4783]:
cogs_ap = get_df_items(income_statement,'Cost of Revenue',start_col).iloc[:,:bl_is_col_diff].apply(abs)
invetories_ap = get_df_items(balance_sheet,'inventories',start_col_balance_sheet)
if len(invetories_ap.value_counts().values) == 0 :
    data = [[0] * len(invetories_ap.columns)]
    invetories_ap = pd.DataFrame(data=data , columns=invetories_ap.columns)
purchases = pd.DataFrame(data=[ [0] * len(invetories_ap.columns) ],columns=invetories_ap.columns.values)
for i , column in enumerate(invetories_ap.columns) :
    if i > 0 :
        currentCol = column
        prevColumn = invetories_ap.columns[i-1]
        inventories_diff = invetories_ap[currentCol] - invetories_ap[prevColumn]
        purchases[currentCol] = cogs_ap[currentCol] + inventories_diff.values    

In [4784]:
account_payables = get_df_items(balance_sheet,'Payables and Accrued Expenses, Current',start_col_balance_sheet)
for i , column in enumerate(balance_sheet.columns):
    if i > 1 :
        prevColumn = balance_sheet.columns[i - 1]
        ap_mean = account_payables.loc[:,prevColumn:column].mean(axis=1)
        turnover_ratio.loc[2:2,column] = calculate_ratio(purchases.loc[:,column],ap_mean).values
turnover_ratio.dropna(how='all' , axis=1,inplace=True)

In [4785]:
#turnover_ratio.drop([start_col_balance_sheet] , inplace=True , axis=1)

In [4786]:
days_turnover_ratio = pd.DataFrame(columns=turnover_ratio.columns)

In [4787]:
days_turnover_ratio[particulars] = ['Receivable turnover days' , 'Inventory turnover days' , 'Payables turnover days']

In [4788]:
days_turnover_ratio.iloc[:,1:] = turnover_ratio.iloc[:,1:].applymap(days_turnover)

In [4789]:
turnover_ratio

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Receivable turnover,3.2,3.38,3.47,3.57,3.76,4.0,3.78,3.83,4.2
1,Inventory turnover,1812.36,2719.76,2868.45,3457.42,2940.26,4473.11,11714.27,13365.38,7161.93
2,Payables turnover,4.57,4.93,5.36,7.56,6.85,6.22,5.49,4.38,3.87


In [4790]:
days_turnover_ratio

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Receivable turnover days,114.06,107.99,105.19,102.24,97.07,91.25,96.56,95.30,86.90
1,Inventory turnover days,0.20,0.13,0.13,0.11,0.12,0.08,0.03,0.03,0.05
2,Payables turnover days,79.87,74.04,68.10,48.28,53.28,58.68,66.48,83.33,94.32


In [4791]:
sc = days_turnover_ratio.columns[1]
receivable_days = get_df_items(days_turnover_ratio,'Receivable turnover days',sc)
inventory_days = get_df_items(days_turnover_ratio,'Inventory turnover days',sc).replace({'-' : 0})
payable_days = get_df_items(days_turnover_ratio,'Payables turnover days',sc)
ccy = inventory_days + receivable_days - payable_days
cash_conversion_cyle = pd.DataFrame(ccy) 


In [4792]:
ccy

,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,34.39,34.08,37.22,54.07,43.91,32.65,30.11,12.0,-7.37


In [4793]:
operating_efficiency_ratio = pd.DataFrame(columns=balance_sheet.columns)

In [4794]:
bl_is_col_diff = len(balance_sheet.columns) - len(income_statement.columns)

In [4795]:
operating_efficiency_ratio[particulars] = ['Total asset turnover' , 'Net fixed asset turnover' , 'Equity turnover']

In [4796]:
operating_efficiency_total_revenue = get_df_items(income_statement,'Total Revenue' , start_col).iloc[:,:bl_is_col_diff]
operating_efficiency_total_assets = get_df_items(balance_sheet,'Total Assets' , start_col_balance_sheet)
operating_efficiency_ppe = get_df_items(balance_sheet,'Net Property, Plant and Equipment' , start_col_balance_sheet)
operating_efficiency_equity = get_df_items(balance_sheet,'Equity Attributable to Parent Stockholders' , start_col_balance_sheet)
for i , column in enumerate(balance_sheet.columns):
    if i > 1 :
        prevColumn = balance_sheet.columns[i-1]
        ta_mean = operating_efficiency_total_assets.loc[:,prevColumn:column].mean(axis=1)
        ppe_mean = operating_efficiency_ppe.loc[:,prevColumn:column].mean(axis=1)
        equity_mean = operating_efficiency_equity.loc[:,prevColumn:column].mean(axis=1)
        operating_efficiency_ratio.loc[0:0,column] = calculate_ratio(operating_efficiency_total_revenue.loc[:,column],ta_mean).values
        operating_efficiency_ratio.loc[1:1,column] = calculate_ratio(operating_efficiency_total_revenue.loc[:,column],ppe_mean).values
        operating_efficiency_ratio.loc[2:2,column] = calculate_ratio(operating_efficiency_total_revenue.loc[:,column],equity_mean).values
operating_efficiency_ratio.dropna(how='all',axis=1,inplace=True)

In [4797]:
operating_efficiency_ratio

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Total asset turnover,1.37,1.34,1.33,1.22,1.17,1.32,1.33,1.3,1.41
1,Net fixed asset turnover,9.04,8.47,8.9,9.88,10.66,12.81,10.06,8.32,9.8
2,Equity turnover,1.86,1.9,1.87,1.56,1.44,1.68,1.81,1.93,2.18


In [4798]:
operating_profitability_ratio_income = pd.DataFrame(columns=income_statement.columns)

In [4799]:
operating_profitability_ratio_income[particulars] = ['Gross profit margin' , 'Operating profit margin', 'Net profit margin' , 'Return on total assets','Return on total equity','Return on owners equity']

In [4800]:
operating_profitability_gross_profit = get_df_items(income_statement,'Gross Profit' , start_col).apply(abs)
operating_profitability_revenue = get_df_items(income_statement,'Total Revenue' , start_col).apply(abs)
operating_income_expense = get_df_items(income_statement,'Operating Income/Expenses' , start_col).apply(abs)
other_operating_income_expense = get_df_items(income_statement,'Other Income/Expense, Operating' , start_col).apply(abs)
EBIT = operating_profitability_gross_profit - (operating_income_expense - other_operating_income_expense)
net_income = get_df_items(income_statement,'Net Income before Extraordinary Items and Discontinued Operations' , start_col)
net_income_owners = get_df_items(income_statement,'Net Income after Non-Controlling/Minority Interests' , start_col)
operating_profitability_total_assets = get_df_items(balance_sheet,'Total Assets' , start_col_balance_sheet)
total_equity = get_df_items(balance_sheet,'Total Equity' , start_col_balance_sheet)
total_owners_equity = get_df_items(balance_sheet,'Equity Attributable to Parent Stockholders' , start_col_balance_sheet)

In [4801]:
operating_profitability_ratio_income.loc[0:0,start_col:] = vertical_analysis(operating_profitability_gross_profit,operating_profitability_revenue).apply(lambda x:round(x,2)).values

In [4802]:
operating_profitability_ratio_income.loc[1:1,start_col:] = vertical_analysis(EBIT,operating_profitability_revenue).apply(lambda x:round(x,2)).values

In [4803]:
operating_profitability_ratio_income.loc[2:2,start_col:] = vertical_analysis(net_income,operating_profitability_revenue).apply(lambda x:round(x,2)).values

In [4804]:
operating_profitability_balance_columns = balance_sheet.loc[:,start_col:].columns

In [4805]:

for i , column in enumerate(operating_profitability_balance_columns) :
    if i > 0 :
        prevColumn = operating_profitability_balance_columns[i-1]
        assets_mean = operating_profitability_total_assets.loc[:,prevColumn:column].mean(axis=1)
        equity_mean = total_equity.loc[:,prevColumn:column].mean(axis=1)
        owners_equity_mean = total_owners_equity.loc[:,prevColumn:column].mean(axis=1)
        operating_profitability_ratio_income.loc[3:3,column] = vertical_analysis(EBIT.loc[:,column],assets_mean).apply(lambda x:round(x,2)).values
        operating_profitability_ratio_income.loc[4:4,column] = vertical_analysis(net_income.loc[:,column],equity_mean).apply(lambda x:round(x,2)).values
        operating_profitability_ratio_income.loc[5:5,column] = vertical_analysis(net_income_owners.loc[:,column],owners_equity_mean).apply(lambda x:round(x,2)).values
        
        
        

In [4806]:
operating_profitability_ratio_income.fillna('-' , inplace=True)

In [4807]:
operating_profitability_ratio_income

,particulars,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Gross profit margin,0.58,0.60,0.55,0.57,0.45,0.44,0.45,0.44,0.47,0.48,0.47
1,Operating profit margin,0.36,0.38,0.33,0.37,0.35,0.34,0.35,0.32,0.32,0.31,0.38
2,Net profit margin,0.22,0.24,0.21,0.22,0.22,0.21,0.22,0.21,0.20,0.20,0.19
3,Return on total assets,-,0.52,0.45,0.49,0.43,0.40,0.46,0.43,0.41,0.43,-
4,Return on total equity,-,0.43,0.39,0.41,0.35,0.30,0.36,0.37,0.38,0.43,-
5,Return on owners equity,-,0.44,0.40,0.42,0.35,0.30,0.36,0.37,0.38,0.44,-


In [4808]:
operating_income_expense / operating_profitability_total_assets

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,0.369054,0.373271,0.399321,0.376001,0.223608,0.219199,0.247923,0.252376,0.264762,0.303143,NaN
